# Welcome
Solution Extraction is a process by which we take a Project Drawdown Solution, in the form of an Excel Workbook, and create a corresponding python solution that implements _most_ of the same functionality.  This notebook will guide you through that process.  See also `Extraction_Guide.md` for more explanation and notes.

The first step is _make a copy of this notebook_.  Give it a name that represents the model you will be working on.  That way it won't collide with other notebooks when you check in or merge fixes.

## Setup


In [1]:
from tools import solution_xls_extract as sxe
from tools import create_expected_zip as cez
from tools import expected_ghost
from tests import test_excel_integration as tei
from solution import factory
from pathlib import Path
import pandas as pd
import openpyxl
import importlib

In [5]:
# Identify where you will be storing your Excel file while you work on it, and what directory the final result will go into.

excelfile = Path("/Users/kunaljoshi/drawdown/Glass_RRS_Model_Commercial-Nov19.xlsm")
outdir = Path("/Users/kunaljoshi/drawdown/solutions/solution/commercialhighperfglass")
outdir.mkdir(parents=True, exist_ok=True)

In [6]:
# If you make changes to the extraction code (or any other code), reload it
# NOTE: This kind of reloading DOES NOT work for solutions themselves, unfortunately.  If you re-generate or modify your solution,
# you have to restart the Jupyter kernel to get it to reload properly.

importlib.reload(sxe)

<module 'tools.solution_xls_extract' from '/Users/kunaljoshi/Drawdown/solutions/tools/solution_xls_extract.py'>

## Extract Code
Exctraction is done by the `sxe.ouput_solution_python_file` function.  This function reads most of the data it needs to extract from the `ScenarioRecord` tab and additional data from the TAM, Adoption and other tabs, and writes them to a solution directory in the form of an `__init__.py` file and a bunch of csv and json files.  All of the solutions in `/solution` were produced this way.

In [7]:
# Expect to see some warnings from openpyxl; these can be ignored.  If there are other warnings, please note them, but they are not necessarily
# a problem.

sxe.output_solution_python_file(outputdir=outdir, xl_filename=str(excelfile))

/Users/kunaljoshi/anaconda3/envs/pd-dev/lib/python3.9/site-packages/openpyxl/reader/drawings.py:59: UserWarning: wmf image format is not supported so the image is being dropped
  warn(msg)
/Users/kunaljoshi/anaconda3/envs/pd-dev/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/Users/kunaljoshi/anaconda3/envs/pd-dev/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/Users/kunaljoshi/anaconda3/envs/pd-dev/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/Users/kunaljoshi/Drawdown/solutions/tools/solution_xls_extract.py:1802: UserWarning: Extraction encountered 48 unknown formulas in values on ScenarioRecord tab
  warnings.warn(f"Extraction encountered {warn_counts['unknown_formula']} unkn

In [7]:
# %debug is your friend.  If the extraction fails with an exception, jump in and see if anything looks wrong

%debug

> c:\projects\project drawdown\solutions\tools\extraction\solution_xls_extract.py(878)write_ad()
    876     f.write("             'Middle East and Africa', 'Latin America', 'China', 'India', 'EU', 'USA'],\n")
    877     f.write("            ['trend', self.ac.soln_pds_adoption_prognostication_trend, ")
--> 878     f.write(q(xls(a, 'L17')) + ",\n")
    879     f.write("             " + q(xls(a, 'L20')) + ", " + q(xls(a, 'L23')) + ", ")
    880     f.write(q(xls(a, 'L26')) + ", " + q(xls(a, 'L29')) + ", ")

ipdb> xls(a, 'L17')
'3rd Poly'
ipdb> q('foo')


If you are working on one of the [Excel import issues](https://github.com/ProjectDrawdown/solutions/issues?q=is%3Aissue+is%3Aopen+label%3A%22Excel+Import%22), please add comments to it describing problems you run into.  And if you have found something that looks like a general problem, please [open a new issue](https://github.com/ProjectDrawdown/solutions/issues/new) for it on the github repo.

I can't overemphasize this: 
> **Finding, researching and reporting issues is hugely valuable for us, even if you don't fully solve them.**


## Load Code / Sniff Test

Once the code has been sucessfully extracted and placed into a directory in `solution/`, all the tools that work with solutions should become available.

In [8]:
# factory.one_solution_scenarios loads a single solution, by name.  It returns a constructor that can construct scenario objects
# for this solution, and a list of the scenario names.

(constructor,scenarios) = factory.one_solution_scenarios("commercialhighperfglass")

In [9]:
# What scenarios did we get?

scenarios

['PDS1-57p2050-based on a 2.75% retrofit rate',
 'PDS3-60p2050-based on an 8% retrofit rate (Integrated)',
 'PDS2-95p2050-Based on 5% Retrofit Rate',
 'PDS3-98p2050-Based on 8% Retrofit Rate',
 'PDS2-59p2050-based on a 5% retrofit rate',
 'PDS3-60p2050-based on an 8% retrofit rate',
 'PDS2-59p2050-based on a 5% retrofit rate (Integrated)',
 'PDS1-94p2050-2.75%retrofit rate-40%initial adoption-Plausible',
 'PDS2-97p2050-5.0-%retrofit rate-40%initial adoption-Drawdown',
 'PDS1-57p2050-based on a 2.75% retrofit rate (integrated)',
 'PDS3-99p2050-8.0-%retrofit rate-40%initial adoption-Optimum',
 'PDS1-89p2050-Based on 2.75% Retrofit Rate']

In [12]:
# Let's build the 2nd one

myscenario = constructor(scenarios[1])

ValueError: Unknown adoption name: 40% Adoption in 2014, 8.0% building stock retrofit rate

In [ ]:
# %debug is your friend.

%debug

> /Users/kunaljoshi/Drawdown/solutions/model/customadoption.py(276)adoption_data_per_region()
    274             result = data['df'].copy()
    275         else:
--> 276             raise ValueError('Unknown adoption name: ' + str(self.soln_adoption_custom_name))
    277         if self.total_adoption_limit is not None:
    278             idx = self.total_adoption_limit.first_valid_index()



## Look at some results

TODO: it would be nice put some examples below, for example showing a little graph of something.

In [28]:
myscenario.c2.co2_mmt_reduced()

World  OECD90  Eastern Europe  Asia (Sans Japan)  Middle East and Africa  Latin America  China  India   EU  USA
Year                                                                                                                      
2014    0.000000     0.0             0.0                0.0                     0.0            0.0    0.0    0.0  0.0  0.0
2015    0.000000     0.0             0.0                0.0                     0.0            0.0    0.0    0.0  0.0  0.0
2016    0.000000     0.0             0.0                0.0                     0.0            0.0    0.0    0.0  0.0  0.0
2017    0.000000     0.0             0.0                0.0                     0.0            0.0    0.0    0.0  0.0  0.0
2018    0.000000     0.0             0.0                0.0                     0.0            0.0    0.0    0.0  0.0  0.0
2019    0.000000     0.0             0.0                0.0                     0.0            0.0    0.0    0.0  0.0  0.0
2020   39.412745     0.0             0.0                0.0                     0.0            0.0    0.0    0.0  0.0  0.0
2021   58.587796     0.0             0.0                0.0                     0.0            0.0    0.0    0.0  0.0  0.0
2022   77.415573     0.0             0.0                0.0                     0.0            0.0    0.0    0.0  0.0  0.0
2023   95.903612     0.0             0.0                0.0                     0.0            0.0    0.0    0.0  0.0  0.0
2024  114.061764     0.0             0.0                0.0                     0.0            0.0    0.0    0.0  0.0  0.0
2025  131.898999     0.0             0.0                0.0                     0.0            0.0    0.0    0.0  0.0  0.0
2026  149.420915     0.0             0.0                0.0                     0.0            0.0    0.0    0.0  0.0  0.0
2027  166.631825     0.0             0.0                0.0                     0.0            0.0    0.0    0.0  0.0  0.0
2028  183.538310     0.0             0.0                0.0                     0.0            0.0    0.0    0.0  0.0  0.0
2029  200.147170     0.0             0.0                0.0                     0.0            0.0    0.0    0.0  0.0  0.0
2030  216.465008     0.0             0.0                0.0                     0.0            0.0    0.0    0.0  0.0  0.0
2031  232.497756     0.0             0.0                0.0                     0.0            0.0    0.0    0.0  0.0  0.0
2032  248.190218     0.0             0.0                0.0                     0.0            0.0    0.0    0.0  0.0  0.0
2033  263.521359     0.0             0.0                0.0                     0.0            0.0    0.0    0.0  0.0  0.0
2034  278.501646     0.0             0.0                0.0                     0.0            0.0    0.0    0.0  0.0  0.0
2035  293.141049     0.0             0.0                0.0                     0.0            0.0    0.0    0.0  0.0  0.0
2036  307.448830     0.0             0.0                0.0                     0.0            0.0    0.0    0.0  0.0  0.0
2037  321.433744     0.0             0.0                0.0                     0.0            0.0    0.0    0.0  0.0  0.0
2038  335.104383     0.0             0.0                0.0                     0.0            0.0    0.0    0.0  0.0  0.0
2039  348.468988     0.0             0.0                0.0                     0.0            0.0    0.0    0.0  0.0  0.0
2040  361.535416     0.0             0.0                0.0                     0.0            0.0    0.0    0.0  0.0  0.0
2041  374.311388     0.0             0.0                0.0                     0.0            0.0    0.0    0.0  0.0  0.0
2042  386.804021     0.0             0.0                0.0                     0.0            0.0    0.0    0.0  0.0  0.0
2043  399.019573     0.0             0.0                0.0                     0.0            0.0    0.0    0.0  0.0  0.0
2044  410.963756     0.0             0.0                0.0                

## Create Test Results

**This step requires the Excel application, and thus only can be run on Windows or Mac.**

Create a clean temporary directory to generate the test set in.  Put (a copy of) your Excel spreadsheet in that directory.

Follow the instructions in `tools/CREATING_EXPECTED_ZIP.md` to create the CSV files in that directory.

In [21]:
# Run the VB macros first!

# Assemble the resulting csv files into the expected_zip file

csvdirectory = Path("C:\\Working\\temp")
cez.create_expected_zip(csvdirectory)

In [24]:
# Move the resulting file where it belongs.

testdirectory = outdir / "testdata"
testdirectory.mkdir(exist_ok=True)

!cp $csvdirectory/expected.zip $testdirectory/expected.zip

## Create the Test

Now we are going to add your new solution to the testing infrastructure.  In an editor, open the file `tests/test_excel_integration.py`.
Scroll down to the bottom, copy one of the test functions there and modify it to fit your new model.  It will look something
like the following, where you subsitute `TESTNAME` with a unique name for your test, and `MODULE` with the name of your solution module (which is the same as the name of the directory it is in).

```
def test_<SOLUTIONNAME>_RRS():
    from solution import <MODULE>
    zipfilename = str(solutiondir.joinpath(
        '<MODULE>', 'testdata', 'expected.zip'))
    zip_f = zipfile.ZipFile(file=zipfilename)
    for scenario in <MODULE>.scenarios.keys():
        obj = <MODULE>.Scenario(scenario=scenario)
        verify = RRS_solution_verify_list(obj=obj, zip_f=zip_f)
        check_excel_against_object(
            obj=obj, zip_f=zip_f, scenario=scenario, verify=verify)
```

Be sure to note whether your model is an RSS model or a Land model and copy the right kind of test!

## Run the Test

The following shell command runs the test you just created (swapping in your solution name, of course)

In [4]:
# You could pytest from the shell, but it is a little more convenient to call the 
# test function directly:
tei.test_Composting_RRS()

Checking PDS-34p2050-May2020
TAM Data
Helper Tables
Emissions Factors
Unit Adoption Calculations


AssertionError: Solution: Composting Scenario: PDS-34p2050-May2020 Unit Adoption Calculations Q135:AA181
DataFrames differ:
Err [2][2] : '126.1526760593272' != '154.5267512826652'
Err [2][3] : '5.77275458555166' != '5.755316301720526'
Err [2][4] : '120.0576454813384' != '84.44148875649192'
Err [2][5] : '10.63625345050044' != '14.852497777486196'
Err [2][6] : '0.461675655780282' != '0.4602810302156386'
Err [2][7] : '9.2038005152678' != '4.6019002576339'
Err [2][8] : '84.5236824402514' != '42.2618412201257'
Err [2][9] : '79.6409' != '39.82045'
Err [2][10] : '45.54' != '22.77'
Err [3][2] : '222.9054876615004' != '251.2795628848384'
Err [3][3] : '8.905047272119651' != '8.887608988288516'


If the test throws an exception, you might be tempted to use %debug to look at it.  Unfortunately the main location where exceptions get thrown has already lost the context of the error.  Instead what you generally have to do is figure out where in the test suite the failure was and add a `breakpoint()` (also known as `import pdb; pdb.set_trace()` to old-schoolers) there, then run it again.   Hint: the error message will probably contain an Excel range, like B91:C137.  This is a good string to search for in `test_excel_integration.py`.

From there, you work your way back to the same questions we were working on above: is this a failure in extraction, model code, the excel workbook, or the test?  Rinse and repeat.

# Tips

## Don't forget to restart the Jupyter Notebook kernel if you have modified code

If you change code you need to either reload the library (the 3nd cell of this notebook) or restart the kernel.

## When comparing to Excel, make sure you've loaded the right Scenario

On the `ScenarioRecord` tab, cell `B9` shows the currently loaded scenario.  When a workbook is first opened, this is usally empty, meaning you don't know which
scenario was last loaded.  Select the scenario you are debugging against from the dropdown, and click on 'Load Scenario'.

## Beautifier for Excel Formulas

Are you looking at an excel formula with five nested `IF(...` expressions?  Try [https://www.excelformulabeautifier.com/](https://www.excelformulabeautifier.com/).  You're welcome.

# You Finished!
Did you get a clean test run?  Hurrah!  You've finished this import task.  Project Drawdown thanks you!